In [14]:
from copy import deepcopy
import pickle
from datetime import datetime as DT
import pandas as pd
import pymc3 as pm
from theano import shared
from pymc_models import PyMCModel
from pymc_models import hs_regression
from pymc_models import bayes_nn_model_ARD_1HL_halfCauchy_hyperpriors as bnn

In [3]:
with open('../PickleJar/DataSets/AphiTrainTestSplitDataSets.pkl', 'rb') as fb:
    datadict = pickle.load(fb)

In [16]:
X_s_train = datadict['x_train_s']
X_s_train_wi = datadict['x_train_wi_s']
y_train = datadict['y_train']
X_s_test = datadict['x_test_s']
X_s_test_wi = datadict['x_test_wi_s']
y_test = datadict['y_test']

In [34]:
X_shared = shared(X_s_train.values)
y_shared = shared(y_train.log10_aphy411.values)

In [32]:
X_shared_wi = shared(X_s_train_wi.values)

In [35]:
hshoe_ = PyMCModel(hs_regression, X_shared, y_shared)

In [33]:
hshoe_wi_ = PyMCModel(hs_regression, X_shared_wi, y_train['log10_aphy411'])

In [36]:
hshoe_.fit(n_samples=2000, cores=4, chains=4, tune=10000,
                    nuts_kwargs=dict(target_accept=0.95))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bias, w, lambda_m, sd_bias, tau]
Sampling 4 chains: 100%|██████████| 48000/48000 [00:51<00:00, 924.45draws/s] 


In [20]:
hshoe_wi_.fit(n_samples=2000, cores=4, chains=4, tune=10000,
             nuts_kwargs=dict(target_accept=0.95))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, bias, w, lambda_m, sd_bias, tau]
Sampling 4 chains: 100%|██████████| 48000/48000 [01:51<00:00, 429.27draws/s]


In [39]:
trace_reg = deepcopy(hshoe_.trace_)
model_reg = deepcopy(hshoe_.model)

In [22]:
trace_reg_wi = deepcopy(hshoe_wi_.trace_)

In [25]:
model_wi = deepcopy(hshoe_wi_.model)

In [26]:
try:
    hshoe_.get_waic()
except UserWarning:
    hshoe_.get_loo()

/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:167: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)
/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:218: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)


In [41]:
pm.waic(trace_reg, model=model_reg)

/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:167: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)
/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:218: UserWarning: For one or more samples the posterior variance of the
        log predictive densities exceeds 0.4. This could be indication of
        WAIC starting to fail see http://arxiv.org/abs/1507.04544 for details
        
  """)


WAIC_r(WAIC=-25.968008219484872, WAIC_se=14.98430342895879, p_WAIC=9.790923125218786, var_warn=1)

In [42]:
pm.loo(trace_reg, model=model_reg)

/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:167: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)


LOO_r(LOO=-25.760587981857693, LOO_se=15.008155718536855, p_LOO=9.894633244032372, shape_warn=0)

In [43]:
pm.loo(hshoe_.trace_, hshoe_.model)

LOO_r(LOO=-25.760587981857693, LOO_se=15.008155718536855, p_LOO=9.894633244032372, shape_warn=0)

In [47]:
X_shared.set_value(X_s_test.values)
y_shared.set_value(y_test.log10_aphy411.values)

In [48]:
pm.loo(hshoe_.trace_, hshoe_.model)

/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:167: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)


LOO_r(LOO=-3.1417467617577586, LOO_se=4.793075455061109, p_LOO=1.3864291111343707, shape_warn=0)

In [53]:
pm.loo(trace_reg, model=hshoe_.model)

/accounts/ekarakoy/anaconda3/envs/bayesian_toa_project/lib/python3.7/site-packages/pymc3/stats.py:167: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.stack(logp)


LOO_r(LOO=-3.1417467617577586, LOO_se=4.793075455061109, p_LOO=1.3864291111343707, shape_warn=0)